In [ ]:
import utils
import numpy as np
import os
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt


pd.set_option('display.expand_frame_repr', False)

%matplotlib inline

results_path = Path("C:/Users/maler/Federico/Università/Master/Tesi/evaluation_results/")

# Scalar Utilities

In [ ]:
utilities_table = utils.load_csv("tables/utilities_table.csv")

In [ ]:
table = utilities_table[utilities_table["Setting"].isin(utils.CONTINUOUS_SETTINGS.intersection(
                                                                utils.LINEAR_SETTINGS
                                                                .union(utils.THRESHOLD_SETTINGS)
                                                                .union(utils.DUAL_THRESHOLD_SETTINGS)
                                                                .union(utils.TARGET_SETTINGS)
                                                                #.union(utils.LINEAR_THRESHOLD_SETTINGS)
                                                                #.union(utils.LINEAR_DUAL_THRESHOLD_SETTINGS)
                                                                )
                                                       )]
table.pivot(index="Setting", columns="Model", values="Utility").head(8)

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Utility"))

# Fixed Environment Evaluation

In [ ]:
fixed_env_unique_non_dom_table = utils.load_csv("tables/fixed_env_unique_non_dom_table.csv")

In [ ]:
table = fixed_env_unique_non_dom_table[fixed_env_unique_non_dom_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(
                                                                utils.LINEAR_SETTINGS
                                                                .union(utils.THRESHOLD_SETTINGS)
                                                                .union(utils.DUAL_THRESHOLD_SETTINGS)
                                                                .union(utils.TARGET_SETTINGS))
                                                       )]

In [ ]:
table.pivot(index="Setting", columns="Model", values="Uniques").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Uniques"))

In [ ]:
table.pivot(index="Setting", columns="Model", values="Non-Dominated").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Non-Dominated"))

In [ ]:
table.pivot(index="Setting", columns="Model", values="Non-Dominated %").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Non-Dominated %"))

In [ ]:
table.pivot(index="Setting", columns="Model", values="Correct Non-Dominated").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Correct Non-Dominated"))

In [ ]:
table.pivot(index="Setting", columns="Model", values="Overall Non-Dominated").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Overall Non-Dominated"))

# Reward Vector Evaluation

In [ ]:
reward_vector_unique_non_dom_table = utils.load_csv("tables/reward_vector_unique_non_dom_table.csv")

In [ ]:
table = reward_vector_unique_non_dom_table[reward_vector_unique_non_dom_table["Setting"].isin(utils.STANDARD_SAMPLING_SETTINGS.intersection(
                                                                utils.LINEAR_SETTINGS
                                                                .union(utils.THRESHOLD_SETTINGS)
                                                                .union(utils.DUAL_THRESHOLD_SETTINGS)
                                                                .union(utils.TARGET_SETTINGS))
                                                       )]
table.pivot(index="Setting", columns="Model", values="Uniques").head()

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Uniques"))

In [ ]:
table.pivot(index="Setting", columns="Model", values="Non-Dominated").head(10)

In [ ]:
utils.convert_to_latex(table.pivot(index="Setting", columns="Model", values="Non-Dominated"))

# Agents' training plot

In [ ]:
STOP

In [ ]:
path = Path("C:/Users/maler/Federico/Università/Master/Tesi/experiments_results/")

ENVS = [
    "cum_linear_threshold_env", "cum_linear_dual_threshold_env", "cum_linear_env", "cum_target_env",
    "cum_dual_threshold_env", "cum_threshold_env", "linear_dual_threshold_env", "linear_threshold_env", "linear_env",
    "target_env", "dual_threshold_env", "threshold_env"
]
MODELS = ["64_64_model", "128_128_64_model", "256_128_128_64_64_model", "512_256_256_128_128_64_model"]


model = "512_256_256_128_128_64_model"
env = "dual_threshold_env"

utils.plot_training_history(path, model, env)

# Computing Tables

In [ ]:
utility_results_path = results_path.joinpath("utility_results/")

utilities_table = utils.utilities_table(utility_results_path)
utilities_table.to_csv("tables/utilities_table.csv")

In [ ]:
fixed_env_results_path = results_path.joinpath("fixed_env_results/")

fixed_env_unique_non_dom_table = utils.uniques_non_dom_table(fixed_env_results_path, fixed_env_flag=True)
fixed_env_unique_non_dom_table.to_csv("tables/fixed_env_unique_non_dom_table.csv")

In [ ]:
reward_vector_results_path = results_path.joinpath("reduced_reward_vector/")

reward_vector_unique_non_dom_table = utils.uniques_non_dom_table(reward_vector_results_path, fixed_env_flag=False)
reward_vector_unique_non_dom_table.to_csv("tables/reward_vector_unique_non_dom_table.csv")

In [ ]:
fixed_env_results_path = results_path.joinpath("fixed_env_results/")

exp_ids = utils.generate_experiments_tuple(fixed_env_results_path)
for exp_id in exp_ids:
    env, model = exp_id[:2]
    
    try:
        l = [np.mean(np.all(result == [1, 0, 0, 0, 0, 0], axis=1)) for result in utils.load_experiment_results(exp_id)]
        if np.any(l):
            print(env, model, l)
    except Exception:
        print(env, model)

In [ ]:
import os

os._exit(00)

In [ ]:
intra_wds_table, inter_wds_table = utils.wds_tables(utility_results)
intra_wds_table.pivot(index="Setting", columns="Model", values="WD").head()

In [ ]:
inter_wds_table.pivot(index="Setting", columns="Model", values="WD").head()